# Rotman Data Science Competition
## Part 1.2: Exploratory Data Analysis of Context of Orders

# Table of Contents
1. [Competition Data Anlaysis](#CompetitionData)

## Part 1. Competition Data Analysis <a name="CompetitionData"></a>
### 1.0 Import Libraries & Load Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
SHOW_GRAPHS = True

In [ ]:
def load_competition_data() -> pd.DataFrame:
    DATA_PATH = "data/mma_mart.csv"
    data = pd.read_csv(DATA_PATH)
    return data

GRAPH_OUT_PATH = "graphs/"

In [ ]:
mma_data = load_competition_data()
mma_data.head()

### 1.1 Examining How Items in an Order Relate to Each Other
To begin, read the data to simply identify whether the items in the order are linked manually, since automatic methods is the aim of this project


In [ ]:
mma_data.head(200)

In [ ]:
prod_sales_count = pd.DataFrame(mma_data.groupby("product_id")["order_id"].count().sort_values(ascending=False))
prod_sales_count.reset_index(inplace=True)
prod_sales_count.columns = ["product_id", "sales_count"]

In [ ]:
# Add column of product names corresponding to the product IDs
id_to_name = mma_data[["product_id", "product_name"]].drop_duplicates().set_index("product_id")
prod_sales_count["product_name"] = prod_sales_count["product_id"].map(id_to_name["product_name"])
prod_sales_count.head(100)

### a) Case Study
First, look for a product that has two very clear ways of substituting. (e.g. apple -> apple juice, apple -> apple extract, apple -> banana)

In [ ]:
# Look for good products for a case study
prod_sales_count[prod_sales_count["product_name"].str.contains("garlic", case=False)]